In [1]:
!pip list

Package                       Version
----------------------------- --------------
absl-py                       0.12.0
alabaster                     0.7.12
albumentations                0.1.12
altair                        4.1.0
appdirs                       1.4.4
argcomplete                   1.12.3
argon2-cffi                   21.1.0
arviz                         0.11.4
astor                         0.8.1
astropy                       4.3.1
astunparse                    1.6.3
atari-py                      0.2.9
atomicwrites                  1.4.0
attrs                         21.2.0
audioread                     2.1.9
autograd                      1.3
Babel                         2.9.1
backcall                      0.2.0
beautifulsoup4                4.6.3
bleach                        4.1.0
blis                          0.4.1
bokeh                         2.3.3
Bottleneck                    1.3.2
branca                        0.4.2
bs4                           0.0.1
CacheControl

In [12]:
import requests
import io

#librerías para graficar
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import os
import time
import sys
                 
plt.rcParams['figure.figsize'] = (16, 9)  #ver graficas grandes 
plt.style.use('ggplot')
#guardar las imagenes y tablas en el cuaderno de jupyter
%matplotlib inline

In [13]:
import tensorflow as tf

In [14]:
print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("GPU is", "available" if tf.config.list_physical_devices('GPU') else "NOT AVAILABLE")

Version:  2.7.0
Eager mode:  True
GPU is available


In [5]:
tf.device('/cpu:0')# OJO: esto es para que tensorflow utilice la GPU (aumenta la velocidad de entrenamiento en un 600%)

In [16]:
fileDL= tf.keras.utils.get_file('Texto Prueba.txt','https://raw.githubusercontent.com/brayan-ahumada/Deep_Learning/main/Texto%20prueba.txt')
texto = open(fileDL, 'rb').read().decode(encoding='utf-8')
texto = texto.lower()

24576/12485 [===========================================================] - 0s 0us/step


In [17]:
print('el texto tiene longitud de:{} caracteres'. format(len(texto)))
vocab = sorted(set(texto))
print('el texto esta compuesto de estos :{} caracteres'. format(len(vocab)))
print(vocab)

el texto tiene longitud de:12251 caracteres
el texto esta compuesto de estos :51 caracteres
['\n', '\r', ' ', '!', '"', ',', '-', '.', '2', '4', ':', ';', '?', '@', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '»', '¿', 'á', 'é', 'í', 'ñ', 'ó', 'ú', 'ü', '–', '“']


In [18]:
char2idx = {u:i for i, u in enumerate(vocab)} # asignamos un número a cada vocablo
idx2char = np.array(vocab)
#-----------revisando las conversiones
#for char,_ in zip(char2idx, range(len(vocab))):
#    print(' {:4s}: {:3d},'.format(repr(char),char2idx[char]))

#pasamos todo el texto a números
texto_como_entero= np.array([char2idx[c] for c in texto])
print('texto: {}'.format(repr(texto[:100])))
print('{}'.format(repr(texto_como_entero[:100])))

texto: 'una carta a garcía\r\nhubo un hombre cuya actuación en la guerra de cuba culmina en los horizontes de '
array([34, 27, 14,  2, 16, 14, 31, 33, 14,  2, 14,  2, 20, 14, 31, 16, 44,
       14,  1,  0, 21, 34, 15, 28,  2, 34, 27,  2, 21, 28, 26, 15, 31, 18,
        2, 16, 34, 38, 14,  2, 14, 16, 33, 34, 14, 16, 22, 46, 27,  2, 18,
       27,  2, 25, 14,  2, 20, 34, 18, 31, 31, 14,  2, 17, 18,  2, 16, 34,
       15, 14,  2, 16, 34, 25, 26, 22, 27, 14,  2, 18, 27,  2, 25, 28, 32,
        2, 21, 28, 31, 22, 39, 28, 27, 33, 18, 32,  2, 17, 18,  2])


Preparacion para usar los datos en la rnn

In [19]:
char_dataset= tf.data.Dataset.from_tensor_slices(texto_como_entero)
#cantidad de secuencia de caracteres
secu_length=150
#creamos secuencias de maximo 100 caractereres
secuencias= char_dataset.batch(secu_length+1, drop_remainder=True)
for item in secuencias.take(10):
  print(repr(''.join(idx2char[item.numpy()])))

'una carta a garcía\r\nhubo un hombre cuya actuación en la guerra de cuba culmina en los horizontes de \r\nmi memoria como culmina un astro en su perihelio.'
'\r\nsucedió que cuando hubo estallado la guerra entre españa y los estados unidos, \r\npalpose clara la necesidad de un entendimiento inmediato entre el pr'
'esidente de la \r\nunión americana y el general calixto garcía ¿cómo hacerlo? hallábase garcía en esos \r\nmomentos dios sabe dónde en alguna serranía perd'
'ida en el interior de la isla. y era \r\nprecisa su colaboración. pero cómo hacer llegar a sus manos un despacho? ¿qué \r\nhacer?\r\nalguien dice al presiden'
'te: "conozco a un hombre llamado rowan. si alguna \r\npersona en el mundo es capaz de dar con garcía es rowan.\r\ncómo el sujeto que lleva por nombre rowan'
' toma la carta, la guarda en una \r\nbolsa que cierra contra su corazón, desembarca a los cuatro días en las \r\ncostas de cuba, desaparece en la selva pri'
'mitiva para reaparecer de nuevo a \r\nlas tres se

Batches (separacion datos por agrupamiento)

In [20]:
#funcion para obtener el conjunto de datos de trainning
def split_input_target(chunk):
  input_text = chunk[:-1]
  target_text= chunk[1:]
  return input_text, target_text

dataset  = secuencias.map(split_input_target)
#el dataset contiene un conjunto de parejas de secuencia de texto
#(con la representación numérica de los caracteres), donde el 
#primer componente de la pareja contiene un paquete con una secuencia 
#de 100 caracteres del texto original y la segunda su correspondiente salida, 
#también de 100 caracteres. )
for input_example, target_example in dataset.take(1):
  print('input data: ', repr(''.join(idx2char[input_example.numpy()])))
  print('Target data: ', repr(''.join(idx2char[target_example.numpy()])))

input data:  'una carta a garcía\r\nhubo un hombre cuya actuación en la guerra de cuba culmina en los horizontes de \r\nmi memoria como culmina un astro en su perihelio'
Target data:  'na carta a garcía\r\nhubo un hombre cuya actuación en la guerra de cuba culmina en los horizontes de \r\nmi memoria como culmina un astro en su perihelio.'


In [21]:
#imprimimos el tensor del dataset
print(dataset)
#Hyper-Parametros para entrenamiento  de una rede neuronal 
#   -los datos se agrupan en batch
BATCH_SIZE= 64
#    -Tamaño de memoria disponible 
BUFFER_SIZE=10000
dataset= dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
print (dataset)
#En el tensor dataset disponemos los datos de entrenamiento
#con agrupamienttos (batches) compuestos de 64 parejas de secuencias 
#de 100 integers de 64 bits que representan el carácter correspondiente 
#en el vocabulario.

<MapDataset shapes: ((150,), (150,)), types: (tf.int64, tf.int64)>
<BatchDataset shapes: ((64, 150), (64, 150)), types: (tf.int64, tf.int64)>


Modelo RNN

In [22]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  #creando el modelo
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.LSTM(rnn_units,
                         return_sequences=True,
                         stateful=True,
                         recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)                               
  ])
  return model
vocab_size= len(vocab)
#dimensiones de los vectores que tendrá la capa.
embedding_dim= 256
#cantidad de neuronas
rnn_units=1024
#creamos nuestra red neuronal RNN
model=build_model(vocab_size=vocab_size,
                  embedding_dim=embedding_dim,
                  rnn_units=rnn_units,
                  batch_size=BATCH_SIZE)
#summary()para visualizar la estructura del modelo
model.summary()
#resultados=
#    -La capa LSTM consta más de 5 millones de parametros)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (64, None, 256)           13056     
                                                                 
 lstm (LSTM)                 (64, None, 1024)          5246976   
                                                                 
 dense (Dense)               (64, None, 51)            52275     
                                                                 
Total params: 5,312,307
Trainable params: 5,312,307
Non-trainable params: 0
_________________________________________________________________


Entrenamiento RNN

In [23]:
#como es un problema de clasificación estándar 
#para el que debemos definir la función de Lossy el optimizador.
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

#En cuanto al optimizador usaremos tf.keras.optimizers.Adam 
#con los argumentos por defecto del optimizador Adam.  
model.compile(optimizer='adam',loss=loss)

Checkpoints

In [24]:
checkpoint_dir='/content/checkpointV2'
checkpoint_prefix= os.path.join(checkpoint_dir,"ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True
)

Entrenamiento

In [25]:
EPOCHS=500
history=model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])
#model.fit(X, y, epochs=50, batch_size=64, callbacks=callbacks_list)

Epoch 1/500
1/1 [==============================] - 7s 7s/step - loss: 3.9326
Epoch 2/500
1/1 [==============================] - 1s 1s/step - loss: 3.8861
Epoch 3/500
1/1 [==============================] - 2s 2s/step - loss: 3.3240
Epoch 4/500
1/1 [==============================] - 1s 521ms/step - loss: 3.5076
Epoch 5/500
1/1 [==============================] - 3s 3s/step - loss: 3.2327
Epoch 6/500
1/1 [==============================] - 1s 1s/step - loss: 3.2297
Epoch 7/500
1/1 [==============================] - 2s 2s/step - loss: 3.1368
Epoch 8/500
1/1 [==============================] - 1s 1s/step - loss: 3.0746
Epoch 9/500
1/1 [==============================] - 1s 1s/step - loss: 3.0230
Epoch 10/500
1/1 [==============================] - 2s 2s/step - loss: 3.0134
Epoch 11/500
1/1 [==============================] - 1s 682ms/step - loss: 3.0053
Epoch 12/500
1/1 [==============================] - 1s 720ms/step - loss: 2.9921
Epoch 13/500
1/1 [==============================] - 3s 3s/step -

Texto nuevo con RNN

In [26]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1,None]))

In [27]:
#funcion para generar texto
def generate_text(model, start_string):
  #definimos cuantos tensores
  num_generate=500
  #convertimos el texto en números
  input_eval=[char2idx[s] for s in start_string]
  input_eval= tf.expand_dims (input_eval,0)
  text_generated = []

  temperature = 0.5  
  #entre más alta la temperatura más creatividad al modelo, pero tambien
  #más errores ortograficos.
  model.reset_states()
  #bucle para generar caracteres, mediante predicciones
  for i in range(num_generate):
    predictions = model(input_eval)
    predictions = tf.squeeze(predictions, 0)
    predictions = predictions / temperature
    predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()
    input_eval= tf.expand_dims([predicted_id],0)
    text_generated.append (idx2char[predicted_id])
  
  return (start_string+ ''.join(text_generated))

In [30]:
print(generate_text(model, start_string=u"buenos dias"))

buenos dias contade: 
que lo coma la carta a garcía cor tuas de las colabiaran en la cila premito almboneciar compañero se lo enseñe a hacer y permanezca a su lado 
para cerciorarse de que lo hacen» éstos pierden constantemente sus puestos
y recibendo algún compañero se lo enseñe a hacer y permanezca a su lado 
para cerciorarse de que lo hacen» éstos pierden constantemente sus puestos
y recibenta en cala prestarmento a lu hará certo a gundorar la osten fuesto para que la ley de saber 
remite a granor


In [29]:
from keras.models import model_from_json
import os
# Serializamos el modelo en forma JSON
model_json = model.to_json()
with open("modelRNN_cuentosV2.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("/content/modelRNN_cuentosV2_pesos.hdf5")
model.save('modelRNN_cuentosV2.h5')
print("modelo salvado en disco")

modelo salvado en disco
